## Imports
utilities.py contains a number of useful functions including loading credentials from a json file, retrieving files from S3, and defining the current list of brains that are associated with the Active Atlas.

In [1]:
import datajoint as dj
import numpy as np
from minio import Minio
import json
import yaml
import sys, os

sys.path.append('./lib')
from utilities import *

## Load Credentials, Connect to Database

In [2]:
%%writefile /data/Github/shapeology_code/scripts/Cell_datajoint/VaultBrain/credFiles_aws.yaml
aws_fp: /data/Github/shapeology_code/scripts/Cell_datajoint/VaultBrain/s3-creds.json
dj_fp: /data/Github/shapeology_code/scripts/Cell_datajoint/VaultBrain/dj_local_conf.json

Writing /data/Github/shapeology_code/scripts/Cell_datajoint/VaultBrain/credFiles_aws.yaml


In [9]:
credFiles='./VaultBrain/credFiles_aws.yaml'

In [6]:
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.config.save_local()

In [3]:
dj.conn()

Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [4]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

## Create tables

In [42]:
Structure.drop()

`kui_diffusionmap`.`structure` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [43]:
@schema
class Structure(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    positive_s3_fp  : varchar(200)  # (str) file path
    negative_s3_fp  : varchar(200)  # (str) file path
    """


In [44]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
all_structures = paired_structures + singular_structures
stack = 'MD589'
s3_fp = 'CSHL_cells_dm/'+stack+'/' #s3://mousebrainatlas-data/
print(s3_fp)

CSHL_cells_dm/MD589/


In [45]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure.insert1(dict(id=n,
                           structure=struc,
                          positive_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_positive.pkl',
                          negative_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_negative.pkl')
                     ,skip_duplicates=False)
    n+=1



Adding 5N to the database

Adding 6N to the database

Adding 7N to the database

Adding 7n to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [5]:
import pandas as pd
structureTable = Structure.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure,positive_s3_fp,negative_s3_fp
0,0,5N,CSHL_cells_dm/MD589/5N/MD589_5N_positive.pkl,CSHL_cells_dm/MD589/5N/MD589_5N_negative.pkl
1,1,6N,CSHL_cells_dm/MD589/6N/MD589_6N_positive.pkl,CSHL_cells_dm/MD589/6N/MD589_6N_negative.pkl
2,2,7N,CSHL_cells_dm/MD589/7N/MD589_7N_positive.pkl,CSHL_cells_dm/MD589/7N/MD589_7N_negative.pkl
3,3,7n,CSHL_cells_dm/MD589/7n/MD589_7n_positive.pkl,CSHL_cells_dm/MD589/7n/MD589_7n_negative.pkl
4,4,Amb,CSHL_cells_dm/MD589/Amb/MD589_Amb_positive.pkl,CSHL_cells_dm/MD589/Amb/MD589_Amb_negative.pkl
5,5,LC,CSHL_cells_dm/MD589/LC/MD589_LC_positive.pkl,CSHL_cells_dm/MD589/LC/MD589_LC_negative.pkl
6,6,LRt,CSHL_cells_dm/MD589/LRt/MD589_LRt_positive.pkl,CSHL_cells_dm/MD589/LRt/MD589_LRt_negative.pkl
7,7,Pn,CSHL_cells_dm/MD589/Pn/MD589_Pn_positive.pkl,CSHL_cells_dm/MD589/Pn/MD589_Pn_negative.pkl
8,8,Tz,CSHL_cells_dm/MD589/Tz/MD589_Tz_positive.pkl,CSHL_cells_dm/MD589/Tz/MD589_Tz_negative.pkl
9,9,VLL,CSHL_cells_dm/MD589/VLL/MD589_VLL_positive.pkl,CSHL_cells_dm/MD589/VLL/MD589_VLL_negative.pkl


In [10]:
client = get_s3_client(credFiles)

In [11]:
report=client.stat_object("mousebrainatlas-data",strucDF['positive_s3_fp'][0])
report.size

17777021964

In [53]:
sys.path.append('../lib')
from utils import run

In [57]:
stack = 'MD589'
yaml_file = 'shape_params.yaml'
scripts_dir = os.environ['REPO_DIR']
struc = '5N'
state = 'positive'
run('python3 {0}/Cell_generator.py {1} {2} {3} {4}'.format(scripts_dir, stack, struc, state, yaml_file))

run cmd= python3 /Users/kuiqian/Github/shapeology_code/scripts//Cell_generator.py MD589 5N positive shape_params.yaml
